In [27]:
import scipy.integrate as integrate
from scipy.integrate import odeint, solve_ivp
import scipy.linalg as la
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, pi, tan, sqrt
import cmath
import sys

plt.rc('text', usetex = True)
plt.rc('font', family = 'serif')

In [28]:
Mav = 47500
Mfus = 46266.1
Mnlg = 145.3
Mmlg = 2*544.3
Ifus = 1.6961e6
Dnm = 14.441
d2 = 13.618
d1 = Dnm - d2
Ampl = 0.01
phi = -2*pi*0.441
V = 7.71667
omega = 2*pi*V
#omega = 111.19499981
#omega = 110.14156318
#omega = 2.31929416
#omega = 8.56023595


print(f"Omega : {omega}")

CA1 = 2*13328.
CA2 = 2962.

KA1 = 2*2895316.
KA2 = 46266.1

Km = 2*3891698
Kn = 1732578

Nddl = 4


#Question 1
M = np.array([[Mmlg, 0, 0, 0],
              [0,Mnlg, 0, 0],
              [0,0, Mfus, 0],
              [0, 0, 0, Ifus]])

invM = np.linalg.inv(M)

C = np.array([
    [CA1   ,  0     , -CA1            ,  d1*CA1               ],
    [0     ,  CA2   , -CA2            , -d2*CA2               ],
    [-CA1  , -CA2   ,  CA1 + CA2      ,  d2*CA2 - d1*CA1      ],
    [d1*CA2, -d2*CA2,  d2*CA2 - d1*CA1,  d1**2*CA1 + d2**2*CA2]
])


K = np.array(
    [[KA1 + Km,   0,         -KA1,             d1 * KA1],
     [0,          KA2 + Kn,  -KA2,             -d2 * KA2],
     [-KA1,       -KA2,      KA1 + KA2,         d2 * KA2 - d1 * KA1],
     [d1 * KA1,   -d2 * KA2, d2 * KA2 - d1 * KA1, d2**2 * KA2 + d1**2 * KA1]]
)

F = np.array([Ampl*Km, Ampl*Kn, 0, 0])
Phase = np.array([phi, 0, 0, 0])

Omega : 48.48526756435349


In [ ]:
#Valeur propre + vecteurs propres
#freqs nats
#8 schémas, 2 par modes propres
#Réponse du sytème ( 1coubre par ddl)
#Amplitudes max et déphases chaque ddl (0 à 120 rad/s)
# Y superposer les amplitudes de forçages
#g réponse du système à la rafale (4 ddl)

LAMBDA, PHI = la.eig(K, M)
print(f"PHI normalisée (norme euclidienne) {PHI}")
Wi = np.sqrt(np.real(LAMBDA))

Mm = np.diag(np.diag(PHI.transpose()@M@PHI))
PHI = PHI@np.sqrt(np.linalg.inv(Mm))
print(f"PHI telle que Phit.M.Phi=I {PHI}")

Mm = PHI.transpose()@M@PHI
Cm = PHI.transpose()@C@PHI

zeta = np.diag(Cm)/2*Wi

Fm = PHI.transpose()*F


PHI normalisée (norme euclidienne) [[ 9.99917481e-01 -1.07955993e-03  3.99283783e-01 -4.81396359e-02]
 [ 7.93951428e-03  9.99999414e-01  1.79275320e-02  2.93697261e-01]
 [-1.00967236e-02 -7.11545130e-05  9.16494669e-01  5.36890216e-01]
 [ 2.26198386e-04 -3.05941922e-05 -1.69878209e-02  7.89413320e-01]]
PHI telle que Phit.M.Phi=I [[ 3.02418133e-02 -8.95590724e-05  2.00838288e-03 -4.65314787e-05]
 [ 2.40125123e-04  8.29588216e-02  9.01748325e-05  2.83885983e-04]
 [-3.05368430e-04 -5.90289800e-06  4.60993479e-03  5.18954811e-04]
 [ 6.84121388e-06 -2.53805962e-06 -8.54481202e-05  7.63042104e-04]]


In [30]:
n = 40001

t = np.linspace(0,100, n)
z0 = np.zeros(2*Nddl)

x = np.zeros((n, Nddl))
y = np.zeros((n, Nddl))

u = np.zeros((n, Nddl))
for i in range(Nddl):
    u[i][:] = F[:]*np.cos(omega*t[i] + Phase[:])

def model(z, t, u, invm, c, k, n):
    xl = np.zeros(n)
    yl = np.zeros(2*n)
    dzdt = np.zeros(2*n)
    for i in range(n):
        xl[i] = z[2*i]
        yl[i] = z[2*i + 1]

    dxdt = yl
    dydt = invm @ (-c @ yl - k @ xl + u)
    for i in range(n):
        dzdt[2*i] = dxdt[i]
        dzdt[2*i + 1] = dydt[i]
    return dzdt

In [ ]:
for i in range(1, n):
    tspan = [t[i-1], t[i]]
    for j in range(Nddl):
        x[i,j] = z[1, 2*j]
        y[i,j = z[1, 2*j + 1]]

In [ ]:
#Tristan est trop coule

PHI telle que Phit.M.Phi=I [[ 3.02418133e-02 -8.95590724e-05  2.00838288e-03 -4.65314787e-05]
 [ 2.40125123e-04  8.29588216e-02  9.01748325e-05  2.83885983e-04]
 [-3.05368430e-04 -5.90289800e-06  4.60993479e-03  5.18954811e-04]
 [ 6.84121388e-06 -2.53805962e-06 -8.54481202e-05  7.63042104e-04]]
